## Importing Modules

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema,PydanticOutputParser
from groq import Groq
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableParallel,RunnableBranch, RunnableLambda



import os
from typing import TypedDict,Annotated,Optional,Literal
from pydantic import BaseModel, EmailStr, Field

import warnings
warnings.filterwarnings("ignore")

load_dotenv()

True

In [2]:
model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

## Sequential Chains

In [4]:

## creating Prompt
prompt = PromptTemplate(
    template = "Generate 3 interesting facts about {topic}",
    input_variables = ["topic"]
)

## create string output parser object
parser = StrOutputParser()

## creating a chain (this is called lanchain expressin language LCEL)
chain = prompt | model | parser
result = chain.invoke({"topic":"biryani"})
print(result)

Here are 3 interesting facts about biryani:

1. **Origins are debated:** While often associated with India, the exact origin of biryani is unclear. Some believe it originated in Persia, others in the Indian subcontinent, and some even trace it back to ancient Arab culinary traditions. 

2. **A royal dish:** Biryani was traditionally a dish reserved for royalty and nobility due to its elaborate preparation and use of expensive ingredients like saffron and meat. 

3. **Regional variations abound:**  Biryani is incredibly diverse, with countless regional variations across India, Pakistan, Bangladesh, and other South Asian countries. Each region boasts its own unique blend of spices, cooking methods, and even types of meat or vegetables used. 


Let me know if you'd like to explore any of these facts further! 



In [10]:
# chain.get_graph().print_ascii()

## Sequential Chain

In [11]:
prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

In [12]:
parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic': 'Unemployment in USA'})

print(result)

Here's a 5-point summary of the text:

1. **Current US Unemployment:** The report highlights the current unemployment rate, labor force participation rate, and number of unemployed individuals, citing data from the Bureau of Labor Statistics.
2. **Unemployment Trends:**  It outlines recent fluctuations in unemployment, including the impact of the COVID-19 pandemic and subsequent economic recovery. Long-term trends show a general decline in unemployment, but with ongoing volatility.
3. **Factors Driving Unemployment:** The report identifies key contributors to unemployment, such as economic cycles, technological change, globalization, demographic shifts, skills gaps, and government policies.
4. **Consequences of Unemployment:**  It emphasizes the negative impacts of unemployment, both on individuals (financial hardship, stress, mental health) and society (strained social welfare, reduced economic output, social unrest).
5. **Potential Solutions:** The report proposes a multi-pronged app

## Parallel Chain

In [24]:
model1 = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )
##

model2 = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="deepseek-r1-distill-llama-70b",
        temperature=0.5 ## this is creative parameter
    )

In [25]:
prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)


In [ ]:
parser = StrOutputParser()

## creating the parallel chain
parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model1 | parser
})

## merging the chain
merge_chain = prompt3 | model2 | parser

chain = parallel_chain | merge_chain


In [27]:
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""


In [28]:
result = chain.invoke({'text':text})

print(result)

## Support Vector Machines (SVMs) 

**What they are:**

* Supervised learning methods for classification, regression, and outlier detection.

**Advantages:**

* **Effective in high dimensions:**  Works well even when features outnumber samples.
* **Memory efficient:** Uses only a subset of training data (support vectors) for decision making.
* **Versatile:**  Different kernel functions can be used for flexibility.

**Disadvantages:**

* **Overfitting risk:**  Careful kernel selection and regularization are needed when features > samples.
* **No direct probability estimates:** Requires expensive cross-validation for probability calculations.

**Input Data:**

* Supports both dense (numpy arrays) and sparse (scipy.sparse) data.
*  Must be trained on the same data type used for prediction.
* **Optimal:** C-ordered numpy arrays (dense) or scipy.sparse.csr_matrix (sparse) with float64 dtype. 

**Quiz:**

1. **Q: What are support vector machines (SVMs) used for?**  
   A: SVMs are used for c

## Conditional Chain

In [31]:
parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)


In [32]:
prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

In [33]:
classifier_chain = prompt1 | model | parser2

In [34]:
prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [ ]:
branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser), ## its lke if else
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

In [39]:
chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

Here are some appropriate responses to positive feedback, depending on the context:

**Formal:**

* Thank you for your kind words. I appreciate your feedback.
* I'm delighted to hear that you're satisfied.
* We strive to provide excellent service, and your feedback is encouraging.

**Informal:**

* Thanks so much! I'm glad you liked it.
* That means a lot to me, thanks!
* Awesome! I'm happy to hear that.

**Adding a personal touch:**

* Thank you! I put a lot of effort into [specific aspect mentioned in feedback], so I'm really glad it showed.
* I appreciate you taking the time to leave feedback. It helps me to know what I'm doing well.
* Your positive feedback is truly motivating!

**When responding to feedback on a product or service:**

* Thank you for your feedback! We're always looking for ways to improve, and we'll take your comments into consideration.
* We're so glad you're enjoying [product/service name]! We're committed to providing our customers with the best possible experi